In [ ]:
import random
from itertools import combinations

# Define card ranks and suits

In [ ]:
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['H', 'D', 'C', 'S']

# Generate a deck of cards

In [ ]:
def generate_deck():
    return [rank + suit for rank in RANKS for suit in SUITS]

# Function to evaluate poker hands with a full evaluation

In [ ]:
def evaluate_hand(hand, community_cards):
    all_cards = hand + community_cards
    all_ranks = [card[0] for card in all_cards]
    all_suits = [card[1] for card in all_cards]
    rank_counts = {rank: all_ranks.count(rank) for rank in all_ranks}
    sorted_ranks = sorted([RANKS.index(rank) for rank in all_ranks], reverse=True)

    is_flush = len(set(all_suits)) == 1
    is_straight = all(sorted_ranks[i] - 1 == sorted_ranks[i + 1] for i in range(len(sorted_ranks) - 1))

    if is_flush and is_straight:
        return 8  # Straight Flush
    elif 4 in rank_counts.values():
        return 7  # Four of a Kind
    elif sorted(rank_counts.values()) == [2, 3]:
        return 6  # Full House
    elif is_flush:
        return 5  # Flush
    elif is_straight:
        return 4  # Straight
    elif 3 in rank_counts.values():
        return 3  # Three of a Kind
    elif list(rank_counts.values()).count(2) == 2:
        return 2  # Two Pair
    elif 2 in rank_counts.values():
        return 1  # One Pair
    else:
        return 0  # High Card

# Expectiminimax algorithm for AI decision-making

In [ ]:
def expectiminimax(state, depth, is_maximizing):
    if depth == 0 or state.is_terminal():
        return state.evaluate()

    if is_maximizing:
        best_value = float('-inf')
        for action in state.get_possible_actions():
            new_state = state.apply_action(action)
            value = expectiminimax(new_state, depth - 1, False)
            best_value = max(best_value, value)
        return best_value
    else:
        best_value = float('inf')
        for action in state.get_possible_actions():
            new_state = state.apply_action(action)
            value = expectiminimax(new_state, depth - 1, True)
            best_value = min(best_value, value)
        return best_value

# Class for poker game state

In [ ]:
class PokerGameState:
    def __init__(self, players, community_cards, pot, current_bet, current_player, active_players):
        self.players = players  # Dictionary of players and their hands
        self.community_cards = community_cards
        self.pot = pot
        self.current_bet = current_bet
        self.current_player = current_player
        self.active_players = active_players  # Players still in the game

    def get_possible_actions(self):
        return ['fold', 'call', 'raise']

    def apply_action(self, action):
        if action == 'fold':
            self.active_players.remove(list(self.players.keys())[self.current_player])
        elif action == 'raise':
            self.pot += 20  # Simulating a raise

        new_state = PokerGameState(self.players, self.community_cards, self.pot, self.current_bet, (self.current_player + 1) % len(self.players), self.active_players)
        return new_state

    def is_terminal(self):
        return len(self.active_players) == 1

    def evaluate(self):
        best_hand = max((evaluate_hand(hand, self.community_cards) for hand in self.players.values()), default=0)
        return best_hand

# Running a multi-player simulation

In [ ]:
def simulate_game():
    deck = generate_deck()
    random.shuffle(deck)

    num_players = 3
    players = {f'Player {i+1}': [deck.pop(), deck.pop()] for i in range(num_players)}
    active_players = list(players.keys())

    community_cards = [deck.pop() for _ in range(5)]

    state = PokerGameState(players=players, community_cards=community_cards, pot=100, current_bet=10, current_player=0, active_players=active_players)

    while not state.is_terminal():
        best_action = expectiminimax(state, depth=3, is_maximizing=True)
        print(f"{list(players.keys())[state.current_player]} chooses action: {best_action}")
        state = state.apply_action(best_action)

    for player, hand in players.items():
        print(f"{player} Hand: {hand}")
    print(f"Community Cards: {community_cards}")
    print(f"Winner: {state.active_players[0]} with best hand evaluation: {evaluate_hand(players[state.active_players[0]], community_cards)}")

simulate_game()